In [7]:
from web3 import Web3
import pandas as pd
import matplotlib.pyplot as plt

# Connect to Ethereum network (Infura or local node)
infura_url = "https://mainnet.infura.io/v3/eth_network"
web3 = Web3(Web3.HTTPProvider(infura_url))

# Check if connected
if web3.is_connected():
    print("Connected to Ethereum")
else:
    print("Failed to connect to Ethereum")


Connected to Ethereum


In [5]:
web3.is_connected()

True

In [8]:
def get_wallet_balance(address):
    # Get the balance of the wallet in Wei (Ethereum's base unit)
    balance = web3.eth.get_balance(address)
    # Convert balance to Ether
    balance_in_ether = web3.fromWei(balance, 'ether')
    return balance_in_ether

def get_transactions(address, start_block=0, end_block='latest', max_tx=10):
    # Get a list of transactions from the address within a block range
    transactions = []
    
    # Retrieve block range
    end_block = web3.eth.block_number if end_block == 'latest' else end_block
    for block_number in range(start_block, end_block):
        block = web3.eth.get_block(block_number, full_transactions=True)
        for tx in block.transactions:
            if tx['from'].lower() == address.lower() or tx['to'].lower() == address.lower():
                transactions.append(tx)
                if len(transactions) >= max_tx:
                    break
        if len(transactions) >= max_tx:
            break
    return transactions

def get_gas_data(transactions):
    gas_data = []
    for tx in transactions:
        gas_used = tx['gas']
        gas_price = tx['gasPrice']
        gas_cost = web3.fromWei(gas_used * gas_price, 'ether')
        gas_data.append(gas_cost)
    return gas_data


In [9]:
def analyze_transactions(transactions):
    # Analyze basic transaction data (count, total gas usage, etc.)
    tx_count = len(transactions)
    gas_data = get_gas_data(transactions)
    total_gas_used = sum(gas_data)
    
    print(f"Total Transactions: {tx_count}")
    print(f"Total Gas Used: {total_gas_used} ETH")
    
    return tx_count, total_gas_used

def analyze_balance(address):
    balance = get_wallet_balance(address)
    print(f"Wallet Balance: {balance} ETH")
    return balance


In [10]:
def plot_gas_usage(gas_data):
    plt.figure(figsize=(10, 5))
    plt.plot(gas_data, label='Gas Cost in ETH')
    plt.xlabel('Transaction Index')
    plt.ylabel('Gas Cost (ETH)')
    plt.title('Gas Usage per Transaction')
    plt.legend()
    plt.show()

def plot_transaction_count(tx_count):
    plt.figure(figsize=(10, 5))
    plt.bar(['Total Transactions'], [tx_count], color='skyblue')
    plt.ylabel('Count')
    plt.title('Number of Transactions')
    plt.show()


In [13]:
def main():
    # Sample wallet address (replace with your own Ethereum wallet address)
    wallet_address = "0xwalletaddress"

    # Fetch transaction data and analyze
    print(f"Analyzing Wallet: {wallet_address}")
    transactions = get_transactions(wallet_address, start_block=13000000, end_block='latest', max_tx=100)
    
    tx_count, total_gas_used = analyze_transactions(transactions)
    balance = analyze_balance(wallet_address)
    
    # Plot gas usage
    gas_data = get_gas_data(transactions)
    plot_gas_usage(gas_data)
    
    # Plot transaction count
    plot_transaction_count(tx_count)

if __name__ == "__main__":
    main()
